In [1]:
import os
import sys
import glob
import numpy
import pandas
import tadpose

# mostly ploting
import ipywidgets
import seaborn as sns
from tqdm.auto import tqdm
from matplotlib import cm, colors
from matplotlib import pyplot as plt

# clustering and pca
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, KMeans
from scipy.ndimage import gaussian_filter1d

### Basic definitions
* create a tadpole object
* configure alignment

In [2]:
# main input required. SLEAP naysis file is expected to be in same folder with ending ".predictions.analysis.h5"
video_fn = "B:/fs3-bifexchgrp/BIF_StaffSci/Christoph/sweengrp/tadpose_examples/Tad1_Take1_oursNOGFP_st59.mp4"

In [3]:
# create Tadpole object
tadpole = tadpose.Tadpole.from_sleap(video_fn)

# create aligner by giving to part names and their correpsonding alignment location
aligner = tadpose.alignment.TadpoleAligner({'tail1' : numpy.array([0, 0.]), 
                                            'heart' : numpy.array([0, 1.])}, scale=False)   
tadpole.aligner = aligner

### Export aligned movie (takes a while) 

In [4]:
#tadpole.export_aligned_movie(dest_height=600, dest_width=200, just_frames=True)

### Define skeleton of interest

In [5]:
cluster_columns = ["tail1", "kneeL", "ankleL", "hindlimbL"] # left limb
# cluster_columns = ["tail1", "tail2", "tail3", "tailtip"] # lail

### Setup

In [6]:
df = tadpole.locations
df_aligned = tadpole.aligned_locations

[[[ 583.99560547  719.71954346]
  [ 495.85687256  651.80908203]]

 [[ 583.98529053  712.22540283]
  [ 492.1763916   643.94378662]]

 [[ 584.03039551  703.85919189]
  [ 491.8130188   639.82946777]]

 ...

 [[1060.3125      883.70074463]
  [1143.92565918  964.27752686]]

 [[1060.30578613  883.65924072]
  [1143.94506836  964.2052002 ]]

 [[1060.30407715  883.65789795]
  [1143.94494629  964.20507812]]]


c:\users\csommer\anaconda3\envs\base_new\lib\site-packages\numpy\linalg\linalg.py:2158: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


### Extract aligned bodypart locations (without liklihood column)

In [7]:
# cryptic one-liner to get x, y coordinates for parts in cluster_columns 
X = df_aligned.loc[:, df_aligned.columns.get_level_values(1)!='likelihood'][cluster_columns]

# the coordinates might contain NaNs (one a part was not detected)
# we have to handle this, since otherwise PCA would fail
X = X.to_numpy()
X_is_nan = numpy.any(numpy.isnan(X), axis=1)

### Do PCA

In [8]:
# subtract mean and scale to unit length 
Xn = (X[~X_is_nan,:] - X[~X_is_nan,:].mean(0)) / X[~X_is_nan,:].std(0)

# PCA with N components
N = 4
Xp = numpy.zeros((len(X), N))

# Xp will contain the PCA components
pca = PCA(n_components=N)
pca.fit(Xn)
Xp[~X_is_nan, :] = pca.transform(Xn)


### Plot randomly selected skeltons with color-coded PC

In [9]:
%matplotlib widget

f, ax = plt.subplots()
ax.set_aspect(1.)

# select PC component to plot
pc = 0

norm = colors.Normalize(vmin=Xp[:, pc].min(), vmax=Xp[:, pc].max())
for rand_ind in numpy.random.randint(X.shape[0], size=200):
    points = X[rand_ind, :]
    load   = Xp[rand_ind, pc]
    color = cm.jet(norm(load)) 
    plt.plot(-points[::2].T, points[1::2].T, ".-",  alpha=0.2, color=color)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Cluster PCA components into K clusters and plot skeltons color-coded cluster asignment

In [10]:
K = 5
km = KMeans(K)
labels = km.fit_predict(Xp)

In [11]:
f, ax = plt.subplots()
ax.set_aspect(1.)
for l in numpy.unique(labels):
    color = next(ax._get_lines.prop_cycler)['color']
    Xl = X[labels==l, :]
    idx = numpy.random.randint(Xl.shape[0], size=200)
    
    points = Xl[idx, :]
    
    plt.plot(-points[:, ::2].T, points[:, 1::2].T, ".-", label=l, alpha=0.1, color=color)
    


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Interactive skelton viewer

In [12]:
%matplotlib widget

def show_skleton_viewer(tadpole, X, Xp, video_shape=(600, 200)):
    """
    Interactive viewer to visualize a skeleton from X together with scalar value "Xp".
    Use left/right keys to go through time
    """
    
    plt.ioff()
    x_view = [-100, 100]
    y_view = [-300, 100]

    slider = ipywidgets.IntSlider(
        description="Time (frame)",
        value=0,
        min=0,
        max=X.shape[0]-1, continuous_update=True,
        style={'min_width': 5000}
    )

    fig, axs = plt.subplots(1,2)
    ax = axs[0]
    ax.set_aspect(1.)

    # which pc-component to vizualize
    pc = 0
    
    # normalize colors for that component
    norm = colors.Normalize(vmin=Xp[:, pc].min(), vmax=Xp[:, pc].max())


    def update_lines(change):
        frame = change.new
        
        points = X[frame, :]
        pc_load   = Xp[frame, pc]
        color = cm.jet(norm(pc_load)) 
        ax.clear()

        gray = tadpole.aligned_image(frame, dest_height=video_shape[0], dest_width=video_shape[1], rgb=False)
        ax.imshow(gray, "gray", extent=(-gray.shape[1]//2, gray.shape[1]//2, -gray.shape[0]//2, gray.shape[0]//2), )

        ax.plot(-points[::2].T, points[1::2].T, ".-",  alpha=1, color=color)
        ax.set_xlim(x_view[0], x_view[1])
        ax.set_ylim(y_view[0], y_view[1])

        grad = Xp[:, 0]

        axs[1].plot(grad, ".-", color="gray")
        axs[1].plot(frame, grad[frame], ".", color="red")
        axs[1].set_xlim(frame-100, frame+100)


        fig.canvas.draw()
        fig.canvas.flush_events()
        

    slider.observe(update_lines, names='value')
    slider.value=1
    
    return ipywidgets.VBox([fig.canvas, slider])

show_skleton_viewer(tadpole, X, Xp)

[[[583.98529053 712.22540283]
  [492.1763916  643.94378662]]]
